In [ ]:
import pandas as pd
import numpy as np
import nltk
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
lemmatizer = WordNetLemmatizer()

In [ ]:

stop_word_dic={}

with open('sample_data/extra_stop.txt','r') as file:
   
    # reading each line    
    for line in file:
   
        # reading each word        
        for word in line.split():
   
            # displaying the words           
            stop_word_dic[word]=word
with open('sample_data/stop_words_english.txt','r') as file:
   
    # reading each line    
    for line in file:
   
        # reading each word        
        for word in line.split():
   
            # displaying the words           
            stop_word_dic[word]=word



closed_class_stop_words = ['a', 'the', 'an', 'and', 'or', 'but', 'about', 'above', 'after', 'along', 'amid', 'among', \
                           'as', 'at', 'by', 'for', 'from', 'in', 'into', 'like', 'minus', 'near', 'of', 'off', 'on', \
                           'onto', 'out', 'over', 'past', 'per', 'plus', 'since', 'till', 'to', 'under', 'until', 'up', \
                           'via', 'vs', 'with', 'that', 'can', 'cannot', 'could', 'may', 'might', 'must', \
                           'need', 'ought', 'shall', 'should', 'will', 'would', 'have', 'had', 'has', 'having', 'be', \
                           'is', 'am', 'are', 'was', 'were', 'being', 'been', 'get', 'gets', 'got', 'gotten', \
                           'getting', 'seem', 'seeming', 'seems', 'seemed', \
                           'enough', 'both', 'all', 'your' 'those', 'this', 'these', \
                           'their', 'the', 'that', 'some', 'our', 'no', 'neither', 'my', \
                           'its', 'his' 'her', 'every', 'either', 'each', 'any', 'another', \
                           'an', 'a', 'just', 'mere', 'such', 'merely' 'right', 'no', 'not', \
                           'only', 'sheer', 'even', 'especially', 'namely', 'as', 'more', \
                           'most', 'less' 'least', 'so', 'enough', 'too', 'pretty', 'quite', \
                           'rather', 'somewhat', 'sufficiently' 'same', 'different', 'such', \
                           'when', 'why', 'where', 'how', 'what', 'who', 'whom', 'which', \
                           'whether', 'why', 'whose', 'if', 'anybody', 'anyone', 'anyplace', \
                           'anything', 'anytime' 'anywhere', 'everybody', 'everyday', \
                           'everyone', 'everyplace', 'everything' 'everywhere', 'whatever', \
                           'whenever', 'whereever', 'whichever', 'whoever', 'whomever' 'he', \
                           'him', 'his', 'her', 'she', 'it', 'they', 'them', 'its', 'their', 'theirs', \
                           'you', 'your', 'yours', 'me', 'my', 'mine', 'I', 'we', 'us', 'much', 'and/or',"wa","ha",",","n't" ,"”" ,"“" ,"’"
                           ]

In [ ]:
df_true= pd.read_csv("sample_data/true_news.csv")
df_fake=pd.read_csv("sample_data/fake_n.csv")
df_pants=pd.read_csv("sample_data/pants_on_fire.csv")

# New Section

In [ ]:
def make_dic():
  for stop in closed_class_stop_words:
    stop_word_dic[stop]=stop           
make_dic()  
def clean_me(string):
  return remove_stop_words(word_tokenize(string))
def remove_stop_words(text):
  result=[]
  for word in text:
    word=word.lower()
    word=lemmatizer.lemmatize(word)
    if word in stop_word_dic:
      continue
    else:
      result.append(word)
  return result

In [ ]:
def clean_data_frameTF_IDF(df,AP,col="Statement"):
  text=df[col]
  clean_text=[]
  id=[]
  Link=[]
  df_clean=pd.DataFrame()
  for index, row in df.iterrows():
    text=row[col]
    res=" ".join(clean_me(text))
    clean_text.append(res)
    if col =="Statement":
      id.append(row['Date'])
      Link.append(row['Link'])
  if(AP):
      for index, row in df_pants.iterrows():
        text=row[col]
        res=" ".join(clean_me(text))
        clean_text.append(res)
        if col =="Statement":
          id.append(row['Date'])
          Link.append(row['Link'])

  if col =="Statement":
    #, "link":Link
    return pd.DataFrame({'Date': id, 'Clean_text': clean_text}),clean_text
  else:
    return pd.DataFrame({'Clean_tweet': clean_text})

df_clean_true=clean_data_frameTF_IDF(df_true,False)
df_clean_false=clean_data_frameTF_IDF(df_fake,True)


In [ ]:
def clean_data_frame(df,AP,col="Statement"):
  text=df[col]
  clean_text=[]
  id=[]
  Link=[]
  df_clean=pd.DataFrame()
  for index, row in df.iterrows():
    text=row[col]
  #text = re.sub(r'@.+', text)
    res=clean_me(text)
    clean_text.append(res)
    if col =="Statement":
      id.append(row['Date'])
      Link.append(row['Link'])
  
  #clean_text.append(clean_me(row[' text']))
  if(AP):
    for index, row in df_pants.iterrows():
      text=row[col]
    #text = re.sub(r'@.+', text)
      res=clean_me(text)
      clean_text.append(res)
      if col =="Statement":
        id.append(row['Date'])
        Link.append(row['Link'])

  if col =="Statement":
    return pd.DataFrame({'Date': id, 'Clean_text': clean_text, "link":Link})
  else:
    return pd.DataFrame({'Clean_tweet': clean_text})

df_clean_true_list=clean_data_frame(df_true,False)
df_clean_false_list=clean_data_frame(df_fake,True)

In [ ]:
corpus=df_clean_false[1]
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
np.random.seed(0)
X = csr_matrix(X)
svd = TruncatedSVD(n_components=1000, n_iter=10, random_state=42)
y=svd.fit_transform(X)

In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=0.5, min_samples=5, metric='cosine').fit(y)

In [ ]:
from numpy.lib.function_base import disp
df=df_clean_false[0]
unique, counts = np.unique(clustering.labels_, return_counts=True)
counts_cluster=(dict(zip(unique, counts)))
sorted_dict = {}
sorted_keys = sorted(counts_cluster, key=counts_cluster.get, reverse=True) 
for w in sorted_keys:
    sorted_dict[w] = counts_cluster[w]
pd.options.display.max_colwidth = 200
print(sorted_dict)
breaker=0
str_result=[]
d=df
for i,x in enumerate(clustering.labels_):
  if(x==12):
    d=df.iloc[[i]]
    str_result.append(df_clean_false_list.iloc[i]['Clean_text'])
    display(d)
    Sep = d.loc[(d['Clean_text'].str.contains('acc'))]
    if(len(Sep["Clean_text"])!=0):
      display(Sep)
    breaker+=1
    if(breaker>300):
      break
print("printing")
print(str_result)



{-1: 4498, 1: 2498, 4: 59, 12: 41, 5: 30, 30: 23, 42: 23, 24: 20, 19: 19, 56: 18, 74: 18, 16: 17, 9: 16, 39: 16, 72: 16, 20: 15, 65: 15, 66: 15, 26: 14, 15: 13, 2: 12, 13: 12, 14: 12, 58: 12, 77: 12, 114: 12, 11: 11, 34: 11, 51: 11, 63: 11, 70: 11, 3: 10, 6: 10, 18: 10, 27: 10, 49: 10, 69: 10, 80: 10, 88: 9, 32: 8, 48: 8, 59: 8, 76: 8, 78: 8, 94: 8, 103: 8, 107: 8, 28: 7, 35: 7, 40: 7, 75: 7, 82: 7, 90: 7, 98: 7, 0: 6, 17: 6, 25: 6, 31: 6, 37: 6, 41: 6, 43: 6, 50: 6, 53: 6, 55: 6, 60: 6, 62: 6, 68: 6, 71: 6, 73: 6, 86: 6, 89: 6, 106: 6, 116: 6, 117: 6, 119: 6, 7: 5, 8: 5, 10: 5, 29: 5, 33: 5, 36: 5, 38: 5, 44: 5, 45: 5, 46: 5, 52: 5, 54: 5, 57: 5, 61: 5, 64: 5, 67: 5, 79: 5, 81: 5, 84: 5, 85: 5, 87: 5, 91: 5, 93: 5, 95: 5, 97: 5, 99: 5, 101: 5, 102: 5, 104: 5, 105: 5, 109: 5, 111: 5, 118: 5, 21: 4, 22: 4, 47: 4, 83: 4, 92: 4, 100: 4, 108: 4, 112: 4, 113: 4, 115: 4, 120: 4, 121: 4, 23: 3, 96: 3, 110: 3}


,Date,Clean_text
269,"November 10, 2021",trump cut social security benefit 3 billion


,Date,Clean_text
434,"October 20, 2021",don comply covid-19 vaccination social security disability won receive money


,Date,Clean_text
434,"October 20, 2021",don comply covid-19 vaccination social security disability won receive money


,Date,Clean_text
1853,"April 5, 2020",italy thrown money street clear message money health danger


...

,Date,Clean_text
7872,"September 1, 2010",ganley elected ‘ cut dickens social security medicare veteran benefit


,Date,Clean_text
7945,"February 3, 2010",congressman mccollum voted dismantle social security


,Date,Clean_text
7982,"July 14, 2009",senate voted week illegal alien access social security benefit


,Date,Clean_text
7982,"July 14, 2009",senate voted week illegal alien access social security benefit


printing
[['trump', 'cut', 'social', 'security', 'benefit', '3', 'billion'], ['don', 'comply', 'covid-19', 'vaccination', 'social', 'security', 'disability', 'won', 'receive', 'money'], ['italy', 'thrown', 'money', 'street', 'clear', 'message', 'money', 'health', 'danger'], ['social', 'security', 'check', 'referred', '‘', 'federal', 'benefit', 'payment'], ['democratic', 'party', 'social', 'security', 'independent', '‘', 'trust', 'fund', 'general', 'fund', 'congress', 'spend', 'eliminated', 'income', 'tax', 'deduction', 'social', 'security', 'fica', 'withholding'], ['president', 'donald', 'trump', 'budget', 'threatened', 'cut', '2', 'trillion', 'social', 'security', 'medicare', 'medicaid'], ['republican', 'congress', 'plotting', 'medicare', 'medicaid', 'social', 'security'], ['fool', 'worse', 'money', 'losing', 'post', 'office', 'money', 'amazon', 'lose', 'fortune', 'changed'], ['housing', 'authority', 'money', 'nycha'], ['republican', 'cut', 'social', 'security', '50', 'percent'], ['do

In [ ]:
from collections import Counter
bag={}

res=" "
for index in str_result:
  
  for el in index:

    if(el in bag):
      bag[el]+=1
    else:
      bag[el]=1
    res=res+" "+str(el)
print(res)
words=res.split(' ')
bigrams = zip(words, words[1:])
bg=[]
for w1 in bigrams:
  bg.append(w1[0]+"_"+w1[1])
counts = Counter(bg)
resy={}
sorted_keys2 = sorted(bag, key=bag.get, reverse=True) 
for w in sorted_keys2:
    resy[w] = bag[w]
sorted_dict1={}
sorted_keys1 = sorted(counts, key=counts.get, reverse=True) 
for w in sorted_keys1:
    sorted_dict1[w] = counts[w]
print(resy)
print(sorted_dict1)

  trump cut social security benefit 3 billion don comply covid-19 vaccination social security disability won receive money italy thrown money street clear message money health danger social security check referred ‘ federal benefit payment democratic party social security independent ‘ trust fund general fund congress spend eliminated income tax deduction social security fica withholding president donald trump budget threatened cut 2 trillion social security medicare medicaid republican congress plotting medicare medicaid social security fool worse money losing post office money amazon lose fortune changed housing authority money nycha republican cut social security 50 percent donald j.trump foundation money money welder money philosopher social security medicare ponzi scheme mark pryor voted social security benefit illegal immigrant texas republican abolishing social security abolishing v.a health care 70 percent federal spending social security medicare medicaid fiscal year 2014 u.s.

In [ ]:
counts